In [ ]:
import { load } from "dotenv";
const env = await load();

In [ ]:
import OpenAI from "openai";

const openai = new OpenAI({
    apiKey: env["AZURE_OPENAI_API_KEY"],
    baseURL: `https://${env["AZURE_OPENAI_API_INSTANCE_NAME"]}.openai.azure.com/openai/deployments/${env["AZURE_OPENAI_API_DEPLOYMENT_NAME"]}`,
    defaultQuery: { 'api-version':  env["AZURE_OPENAI_API_VERSION"] },
    defaultHeaders: { 'api-key': env["AZURE_OPENAI_API_KEY"] },
});

In [ ]:
const result = await openai.chat.completions.create({
    // model: 'gpt-3.5-turbo',
    model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
    messages: [{ role: 'user', content: 'Say hello!' }],
  });
  console.log(result.choices[0]!.message?.content);

In [ ]:
function getCurrentWeather({ location, unit="fahrenheit"}){
   const  weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return JSON.stringify(weather_info);
}

In [ ]:
const tools = [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: {
          type: "object",
          properties: {
            location: {
              type: "string",
              description: "The city and state, e.g. San Francisco, CA",
            },
            unit: { 
              type: "string", 
              enum: ["celsius", "fahrenheit"],
              description: "The unit of temperature"
            },
          },
          required: ["location", "unit"],
        },
      },
    }
]

In [ ]:
 const messages = [
    {
        "role": "user",
        // "content": "北京的天气怎么样"
        "content": "What's the weather like in Redmond?"

    }
]

const result = await openai.chat.completions.create({
    // model: 'gpt-3.5-turbo',
    model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
    messages,
    tools
  });
  console.log(result);

In [ ]:
result.choices[0]

In [ ]:

const messages = [
    {
        "role": "user",
        "content": "你好"
    }
]

const result = await openai.chat.completions.create({
    // model: 'gpt-3.5-turbo',
    model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
    messages,
    tools,
    tool_choice: {
        type: "function",
        function: {
           name: "getCurrentWeather"
        }
    }
  });

In [ ]:

const functions = {
    "getCurrentWeather": getCurrentWeather
  }

const functionInfo = result.choices[0].message.tool_calls[0].function
const functionName = functionInfo.name;
const functionParams = functionInfo.arguments

const functionResult = functions[functionName](functionParams);

console.log(functionResult);

In [ ]:
function getCurrentTime({ format = "iso" } = {}) {
    let currentTime;
    switch (format) {
        case "iso":
            currentTime = new Date().toISOString();
            break;
        case "locale":
            currentTime = new Date().toLocaleString();
            break;
        default:
            currentTime = new Date().toString();
            break;
    }
    return currentTime;
}

const tools = [
    {
        type: "function",
        function: {
            name: "getCurrentTime",
            description: "Get the current time in a given format",
            parameters: {
                type: "object",
                properties: {
                    format: {
                        type: "string",
                        enum: ["iso", "locale", "string"],
                        description: "The format of the time, e.g. iso, locale, string",
                    },
                },
                required: [],
            },
        },
    },
    {
        type: "function",
        function: {
          name: "getCurrentWeather",
          description: "Get the current weather in a given location",
          parameters: {
            type: "object",
            properties: {
              location: {
                type: "string",
                description: "The city and state, e.g. San Francisco, CA",
              },
              unit: { type: "string", enum: ["celsius", "fahrenheit"] },
            },
            required: ["location", "unit"],
          },
        },
      }
];

In [ ]:

const messages = [
    { role: "user", content: " 上海 新疆 这三个城市的天气如何?" },
]

const result = await openai.chat.completions.create({
    // model: 'gpt-3.5-turbo',
    model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
    messages,
    tools
  });


In [ ]:
result.choices[0]

In [ ]:

messages.push(result.choices[0].message)

const functions = {
    "getCurrentWeather": getCurrentWeather
  }

const cell = result.choices[0].message.tool_calls[0]
const functionInfo = cell.function
const functionName = functionInfo.name;
const functionParams = functionInfo.arguments
const functionResult = functions[functionName](functionParams);

console.log(messages);
messages.push({
  tool_call_id: cell.id,
  role: "tool",
  name: functionName,
  content: functionResult,
}); 

const response = await openai.chat.completions.create({
  // model: 'gpt-3.5-turbo',
  model: env["AZURE_OPENAI_API_DEPLOYMENT_NAME"],
  messages,
});
console.log(response);

In [ ]:
response.choices[0].message

In [ ]:
messages